## How to Run

1. Run the steps in the main README.

2. For MPC authentication, get your subscription key from [here](https://planetarycomputer.developer.azure-api.net/profile) and run `planetarycomputer configure` in terminal and paste is there. Further instructions [here](https://planetarycomputer.microsoft.com/docs/concepts/sas/#:~:text=data%20catalog.-,planetary%2Dcomputer%20Python%20package,-The%20planetary%2Dcomputer).

3. Set your required search time window, image size, and number of features in the parameters section.

4. Run the notebook.

#### To Do:
- Can we go older than 2013? No Landsat 7 on this endpoint...
- Add column to output CSV to record why a no features where returned for a requested point
- Read parameters from .yml/.json
- Match dimensions of composited vs raw image tensors

In [1]:
import warnings
import os
import gc
from pathlib import Path

from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader

import numpy as np
import pandas as pd
import geopandas as gpd
import dask_geopandas as dask_gpd
from dask.distributed import Client
from dask_gateway import GatewayCluster

import matplotlib.pyplot as plt

In [2]:
from custom.mosaiks_points import load_points_gdf

from custom.mpc_imagery import (
    sort_by_hilbert_distance,
    fetch_stac_items, 
    CustomDataset
)
from custom.models import featurize, RCF
from custom.mosaiks_data import save_features_to_parquet

In [3]:
warnings.filterwarnings(action="ignore", category=RuntimeWarning)
warnings.filterwarnings(action="ignore", category=UserWarning)

In [4]:
RASTERIO_BEST_PRACTICES = dict(  # See https://github.com/pangeo-data/cog-best-practices
        CURL_CA_BUNDLE="/etc/ssl/certs/ca-certificates.crt",
        GDAL_DISABLE_READDIR_ON_OPEN="EMPTY_DIR",
        AWS_NO_SIGN_REQUEST="YES",
        GDAL_MAX_RAW_BLOCK_CACHE_SIZE="200000000",
        GDAL_SWATH_SIZE="200000000",
        VSI_CURL_CACHE_SIZE="200000000",
)
os.environ.update(RASTERIO_BEST_PRACTICES)

In [5]:
ROOT = Path.cwd().parent
DATA_ROOT = ROOT / "data"

## Set parameters

Note: Earliest L8 imagery is April 2013 so we pick the least cloudy (or least cloudy composite) image from the 1 year window after this date. Latest imagery is March 2022, so we take 1 year window prior to this date.

In [38]:
data_label = "TEMP_2013_exact_2400m_v4000_L8"

In [39]:
satellite = "landsat-8-c2-l2" #"sentinel-2-l2a"
resolution = 30
bands = [
    # "SR_B1", # Coastal/Aerosol Band (B1)
    "SR_B2",  # Blue Band (B2)
    "SR_B3",  # Green Band (B3)
    "SR_B4",  # Red Band (B4)
    "SR_B5",  # Near Infrared Band 0.8 (B5)
    "SR_B6",  # Short-wave Infrared Band 1.6 (B6)
    "SR_B7",  # Short-wave Infrared Band 2.2 (B7)
]

In [2]:
search_start = "2013-04-01" #"2021-04-01" #"2015-11-01"
search_end = "2014-03-31" #"2022-03-31" #"2015-11-01"
# 1500m buffer results in 100x100px images for Landsat's 30m resolution
BUFFER_DISTANCE = 1200 # gives 80x80px pics
NUM_FEATURES = 4000

# for image fetching
min_image_edge = 30

## Load point coordinates to fetch images for

In [9]:
# static (single-machine workers)
client = Client(n_workers=8)
print(client.dashboard_link)

/user/amirali1376@gmail.com/proxy/8787/status


In [10]:
# # adaptive Dask client (multi-machine)
# cluster = GatewayCluster()
# client = cluster.get_client()
# cluster.adapt(minimum=4, maximum=24)
# print(cluster.dashboard_link)

In [11]:
# cluster.close()

In [12]:
filepath = DATA_ROOT / "01_preprocessed/mosaiks_request_points" / "urb_rur_request_points.csv" #  "INDIA_SHRUG_request_points.csv"
points_gdf = load_points_gdf(filepath)
# has 99,897 points

In [13]:
NUM_POINTS = len(points_gdf)
print("No. points loaded:", NUM_POINTS)

No. points loaded: 99897


In [14]:
points_gdf = sort_by_hilbert_distance(points_gdf)

In [15]:
# TEMP
points_gdf = points_gdf[:10]
points_gdf

,Lat,Lon,geometry,hilbert_distance
91057,13.754878,74.634486,POINT (74.63449 13.75488),174607565
91056,13.804878,74.634486,POINT (74.63449 13.80488),174624606
91062,13.854878,74.634486,POINT (74.63449 13.85488),174633202
89896,13.954878,74.584486,POINT (74.58449 13.95488),174676976
89898,13.954878,74.634486,POINT (74.63449 13.95488),174693283
91055,13.904878,74.634486,POINT (74.63449 13.90488),174706860
91061,13.904878,74.684486,POINT (74.68449 13.90488),174731696
91060,13.954878,74.684486,POINT (74.68449 13.95488),174741557
90845,13.954878,74.734486,POINT (74.73449 13.95488),174757818
91065,13.904878,74.734486,POINT (74.73449 13.90488),174773397


Convert to DaskGeoDataFrame for parallelization

In [16]:
NPARTITIONS = 250
points_dgdf = dask_gpd.from_geopandas(points_gdf, npartitions=NPARTITIONS, sort=False)

del points_gdf
gc.collect()

41

## Get the imagery around each point

### Get image refs

Get stac_item references to images (or least cloudy image) that corresponds to each point

In [27]:
# `meta` is the expected output format: an empty df with correct column types
meta = points_dgdf._meta
meta = meta.assign(stac_item=pd.Series([], dtype="object"))

points_gdf_with_stac = points_dgdf.map_partitions(
    fetch_stac_items, 
    satellite=satellite,
    search_start=search_start,
    search_end=search_end,
    stac_output="all",
    meta=meta)

points_gdf_with_stac = points_gdf_with_stac.compute()

In [28]:
points_gdf_with_stac.head(2)

,Lat,Lon,geometry,hilbert_distance,stac_item
91057,13.754878,74.634486,POINT (74.63449 13.75488),174607565,"[<Item id=LC08_L2SP_146051_20140329_02_T1>, <I..."
91056,13.804878,74.634486,POINT (74.63449 13.80488),174624606,"[<Item id=LC08_L2SP_146051_20140329_02_T1>, <I..."


In [29]:
stac_item_list[0]

ID: LC08_L2SP_146050_20140313_02_T1
"Bounding Box: [73.42619150783845, 13.39531443265443, 75.5521098010548, 15.50460556734557]"
Datetime: 2014-03-13 05:23:21.906998+00:00
datetime: 2014-03-13T05:23:21.906998Z
platform: landsat-8
"proj:bbox: [331185.0, 1481385.0, 559215.0, 1714215.0]"
proj:epsg: 32643
description: Landsat Collection 2 Level-2 Surface Reflectance Product
"instruments: ['oli', 'tirs']"
eo:cloud_cover: 0.0
view:off_nadir: 0


In [30]:
# save the found STAC image ref to file
points_gdf_with_stac.to_pickle(DATA_ROOT / "01_preprocessed/mosaiks_features" / f"TEMP_latlons_and_stacs_{data_label}.pkl")

In [31]:
stac_item_list = points_gdf_with_stac.stac_item.tolist()
points_list = points_gdf_with_stac[["Lon", "Lat"]].to_numpy()

### Setup PyTorch data loader

In [32]:
dataset = CustomDataset(
    points_list,
    stac_item_list, 
    buffer=BUFFER_DISTANCE,
    bands=bands,
    resolution=resolution
)

batch_size = 1 # increase this?
num_workers = os.cpu_count() # AWS can handle *2 since 32GB ram
dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=lambda x: x,
    pin_memory=False,
)

    ### The following clashes with Dask workers! ###
    # num_workers=num_workers,
    # persistent_workers=True,

### Clear memory

In [33]:
# del client
# del points_gdf_with_stac
# del stac_item_list
# # del points_list
# gc.collect()

### Check images

In [1]:
# for images in dataloader:
#     for image in images:        
#         array = np.array(image[:3])
#         array = np.flip(array)
#         reshaped_array = np.swapaxes(array, 0, 2)
#         plt.imshow(reshaped_array)
#         plt.show()

## Define featurization model and apply to images

In [36]:
DEVICE = torch.device("cuda") # change to "cuda" when using GPU
MODEL = RCF(NUM_FEATURES).eval().to(DEVICE)

### Apply featurization to images

Full image should be 100x100 pixels (3km^2 at 30m/px), but we can receive smaller images if an input point happens to be at the edge of a scene. To deal with these we crudely drop all images where the spatial dimensions aren't both greater than `min_image_edge` pixels.

In [37]:
X_all = np.full([NUM_POINTS, NUM_FEATURES], np.nan, dtype=float)

i = 0
end_index = 0
for images in tqdm(dataloader):
    for image in images:
        if image is not None:
            if image.shape[1] >= min_image_edge and image.shape[2] >= min_image_edge:
                mosaiks_features = featurize(image, MODEL, DEVICE)
                X_all[i] = mosaiks_features
            else:
                # if image size is too small
                pass
        else:
            # if we do have not found an image for some point
            pass
        
        # save to file every 10k points
        if (i!=0 and i%10000==0) or i==NUM_POINTS:
            start_index = end_index
            end_index = i
            X_chunk = X_all[start_index:end_index]

            print(f"Saving {start_index} to {end_index-1}...")
            
            save_features_to_parquet(
                array=X_chunk, 
                points_list=points_list, 
                start_index=start_index, 
                end_index=end_index, 
                output_folder_path=DATA_ROOT/"01_preprocessed/mosaiks_features/", 
                filename_prefix="temp_"+data_label,
            )
            
            del X_chunk
            gc.collect()

        i += 1

  0%|          | 0/10 [00:00<?, ?it/s]

### Save to file (gzipped parquet)

In [ ]:
save_features_to_parquet(
    array=X_all, 
    points_list=points_list,
    output_folder_path=DATA_ROOT/"01_preprocessed/mosaiks_features/", 
    filename_prefix=data_label,
)